# Sail routing

In [1]:
import json
import datetime
import codecs
import glob

from IPython.display import clear_output
import numpy as np
import ipywidgets as widgets
from scipy.interpolate import RegularGridInterpolator, LinearNDInterpolator
from scipy.optimize import minimize, differential_evolution
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from pyproj import Geod, enums
import xarray as xr

# Conversion between meters and nautical miles
NM = 1852.0
# A tiny value to prevent division by zero
EPS = 0.01
# A small value for speed "into the wind" an "on land"
SLOW = 0.5
# Geodetic system
geoid = Geod(ellps="WGS84")


## Set up start point, end point and the map

In [2]:
# Starting POSITION: Rønne, Bornholm, Denmark
STA_LAT = 55.09391858131019
STA_LON = 14.67659876334345
START = np.array([[STA_LON], [STA_LAT]])

# END POSITION: Sassnitz, Ruegen, Germany
END_LAT = 54.41598502647239
END_LON = 13.627842253135594
END = np.array([[END_LON], [END_LAT]])

# Compute region of map
buffer = 0.2
MIN_LON = min(STA_LON, END_LON) - buffer * abs(STA_LON - END_LON)
MIN_LAT = min(STA_LAT, END_LAT) - buffer * abs(STA_LAT - END_LAT)
MAX_LON = max(STA_LON, END_LON) + buffer * abs(STA_LON - END_LON)
MAX_LAT = max(STA_LAT, END_LAT) + buffer * abs(STA_LAT - END_LAT)

# Create map
m = Basemap(
    llcrnrlon=MIN_LON,
    llcrnrlat=MIN_LAT,
    urcrnrlon=MAX_LON,
    urcrnrlat=MAX_LAT,
    resolution="f",
    projection="merc",
)


## Wind 

In [ ]:
# Read GRIB file
files = glob.glob("weather/*.grb")
for file in files:
    ds = xr.open_dataset(file, engine="cfgrib")

# Select relevant section of GRIB file
ds = ds.where(
    (ds["longitude"] > MIN_LON)
    & (ds["longitude"] < MAX_LON)
    & (ds["latitude"] > MIN_LAT)
    & (ds["latitude"] < MAX_LAT),
    drop=True,
)

# Build interpolation tables
wind_time = ds["step"].to_numpy().astype(float) / 3600.0e9 - 6.0
wind_lons = ds["longitude"].to_numpy()
wind_lats = ds["latitude"].to_numpy()
wind_u10 = ds["u10"].to_numpy()
wind_v10 = ds["v10"].to_numpy()
wind_u = RegularGridInterpolator(
    (wind_time, wind_lats, wind_lons), wind_u10, bounds_error=False, fill_value=0.0
)
wind_v = RegularGridInterpolator(
    (wind_time, wind_lats, wind_lons), wind_v10, bounds_error=False, fill_value=0.0
)

# Wind function
def wind(time, lon, lat):
    t = time
    u = wind_u((t, lat, lon))
    v = wind_v((t, lat, lon))
    mag = np.sqrt(u**2 + v**2)
    dir = np.rad2deg(np.arctan(-v / (u + EPS))) - 90.0
    return [dir, mag]


## Waypoints 

In [ ]:
N = 5
wp = geoid.npts(STA_LON, STA_LAT, END_LON, END_LAT, N)
waypoints = np.array([[pos[0] for pos in wp], [pos[1] for pos in wp]])
initial_waypoints = np.hstack([START, waypoints, END])

# Create initial population
POPSIZE = 15
noise = (np.random.rand(POPSIZE, len(waypoints.ravel())) - 0.5)
initial_population = np.tile(waypoints.ravel(), (POPSIZE, 1)) + noise 

## Load polar chart

In [ ]:
# Load boats
boats = json.load(codecs.open("boats.json", "r", "utf-8-sig"), strict=False)
boat = list(filter(lambda b: b["boat"]["type"] == "XC-45", boats))[0]

# Align measured angles and wind speeds in a meshgrid for interpolation
min_angle = max(boat["vpp"]["beat_angle"])
max_angle = 180.0
a = np.array([min_angle] + boat["vpp"]["angles"] + [max_angle], dtype=float)
s = np.array(boat["vpp"]["speeds"], dtype=float)
angles, windspeeds = np.meshgrid(a, s)

# Fill table of velocities for interpolation
vel = np.zeros_like(angles)
vel[:, 0] = boat["vpp"]["beat_vmg"]
for i, angle in enumerate(boat["vpp"]["angles"]):
    vel[:, i + 1] = boat["vpp"][f"{angle}"]
vel[:, -1] = boat["vpp"]["run_vmg"]

# Build a look up table for interpolation of boat speed
lut = RegularGridInterpolator((s, a), vel, bounds_error=False, fill_value=SLOW)


In [ ]:
# Plot polar diagram of the boat
fig, ax = plt.subplots(subplot_kw={"projection": "polar"})
ax.set_theta_zero_location("N")
ax.set_thetamin(0)
ax.set_thetamax(180)
for a, v in zip(angles, vel):
    ax.plot(np.deg2rad(a), v, "o-")
plt.legend([f"{ws} kn" for ws in boat["vpp"]["speeds"]], loc="lower right")
plt.title(boat["boat"]["type"] + " '" + boat["name"] + "'")
plt.show()

In [ ]:
def velocity(course, wind_dir, wind_mag):
    angle = np.abs(((course + 360.0) % 360.0 - wind_dir))
    if angle > 180:
        angle = 360.0 - angle
    return lut((wind_mag, angle))


## Compute travel time 

In [ ]:
def compute_path(waypoints, opt=False):
    # Build waypoints from start point, end point and optimization variables
    wpts = np.hstack([START, waypoints.reshape(2, -1), END])
    lons = wpts[0, :]
    lats = wpts[1, :]
    lengths = geoid.line_lengths(lons, lats)
    # Initialize integrated time
    int_time = 0
    # Intialize arrays to store path
    if not opt:
        t = []
        v = []
        la = []
        lo = []

    # Iterate over waypoint segments
    for i in range(1, len(lons)):
        # Integration steps along segment are 500m long, but at least 2 per path segment
        del_s = min(500, lengths[i - 1] / 3.0)
        flags = enums.GeodIntermediateFlag.AZIS_KEEP
        intp = geoid.inv_intermediate(
            lons[i - 1],
            lats[i - 1],
            lons[i],
            lats[i],
            del_s=del_s,
            terminus_idx=0,
            flags=flags,
        )
        ds = intp.del_s / NM

        # Iterate over integration points along segment
        for lon, lat, azi in zip(intp.lons, intp.lats, intp.azis):
            x, y = m(lon,lat)
            if m.is_land(x, y):
                # on land we penalize the solution with a very low speed
                vel = SLOW
            else:
                # at sea, we compute the speed from wind and polar diagram
                wind_dir, wind_mag = wind(int_time, lon, lat)
                vel = velocity(azi, wind_dir, wind_mag)
            # Integrate time
            int_time += ds / vel
            # Store path
            if not opt:
                t.append(int_time)
                v.append(vel)
                la.append(lat)
                lo.append(lon)
    if opt:
        return int_time
    else:
        return {
            "time": np.array(t),
            "velocity": np.array(v),
            "lons": np.array(lo),
            "lats": np.array(la),
        }


# import cProfile
# import pstats
# cProfile.run('compute_path(waypoints, start, end)', 'stats')
# p = pstats.Stats('stats')
# p.sort_stats(pstats.SortKey.TIME).print_stats(10)


## Optimization

In [ ]:
def callback(waypoints, convergence):
    clear_output(wait=True)
    wpts = np.hstack([START, waypoints.reshape(2, -1), END])
    m.drawcoastlines()
    m.fillcontinents()
    m.drawgreatcircle(
        STA_LON, STA_LAT, END_LON, END_LAT, del_s=1, color="k", linestyle=":"
    )
    # Plot waypoints
    x, y = m(wpts[0, :], wpts[1, :])
    m.plot(x, y, color="k", marker="X")
    plt.title(convergence)
    plt.show()


bounds = N * [(MIN_LON, MAX_LON)] + N * [(MIN_LAT, MAX_LAT)]
opt = differential_evolution(
    compute_path,
    bounds,
    args=(True,),
    disp=True,
    maxiter=100,
    init=initial_population,
    callback=callback,
)
waypoints = opt.x.reshape(2, -1)

path = compute_path(waypoints)
full_waypoints = np.hstack([START, waypoints, END])
dist = geoid.line_length(full_waypoints[0, :], full_waypoints[1, :]) / NM

# Print resulting calculation
td = datetime.timedelta(hours=path["time"][-1])
td = ":".join(str(td).split(":")[:2])
print(f"The travel time is {td} h for {dist:.2f} nautical miles.")


## Plot path

In [ ]:
def plot_func(time):
    # Clear plot
    plt.clf()

    # Draw map
    m.drawcoastlines()
    m.fillcontinents()
    m.drawgreatcircle(
        STA_LON,
        STA_LAT,
        END_LON,
        END_LAT,
        del_s=1,
        color="k",
        linestyle=":",
    )

    # Print wind magnitude
    wlons, wlats = np.meshgrid(wind_lons, wind_lats)
    xx, yy = m(wlons, wlats)
    wdir, wmag = wind(time, wlons, wlats)
    cont = m.contourf(xx, yy, wmag, cmap="magma", alpha=0.5)
    plt.colorbar(cont, label="Windspeed in kn")

    # Plot wind barbs
    all_lons = np.linspace(MIN_LON, MAX_LON, 8)
    all_lats = np.linspace(MIN_LAT, MAX_LAT, 8)
    alons, alats = np.meshgrid(all_lons, all_lats)
    xx, yy = m(alons, alats)
    wdir, wmag = wind(time, alons, alats)
    vx = -wmag * np.sin(np.deg2rad(wdir))
    vy = -wmag * np.cos(np.deg2rad(wdir))
    m.barbs(xx, yy, vx, vy)

    # Plot path
    idx = np.where(path["time"] < time)
    x, y = m(path["lons"][idx], path["lats"][idx])
    velocity = path["velocity"][idx]
    vmin = np.min(path["velocity"])
    vmax = np.max(path["velocity"])
    scat = m.scatter(x, y, c=velocity, vmin=vmin, vmax=vmax)
    plt.colorbar(scat, label="Boatspeed in kn")

    # Plot waypoints
    x, y = m(full_waypoints[0, :], full_waypoints[1, :])
    m.scatter(x, y, color="k", marker="X")

    # Draw plot
    plt.draw()

widgets.interact(
    plot_func,
    time=widgets.FloatSlider(
        value=path["time"][-1], min=0.1, max=path["time"][-1], step=0.1
    ),
)


In [ ]:
fig = plt.figure()
anim = animation.FuncAnimation(fig, func=plot_func,frames=np.linspace(0, path["time"][-1], 60))
anim.save("animation.gif")